#Overview


This rough notebook implements an example of a text bottleneck model. It is meant to be run as a google colab notebook. It currently downloads the cebab dataset and trains a TBM over it as an example.

# Imports and downloads

In [2]:
%%capture
#Downloading necessary packages
!pip install openai     #Finetuning GPT-3
!pip install transformers #For GPT-2 tokenizer for cost calculations
!pip install swifter      #Useful for evaluation, gives a loading bar and parallelizes calls to API

In [3]:
import pandas as pd
import numpy as np
#Expanding display size for convenience
pd.options.display.max_colwidth = 1000

import random
random.seed(42)
import pandas as pd
import numpy as np
import json
import openai
import os
import swifter
from tenacity import retry, stop_after_attempt, wait_random_exponential, retry_if_exception_type
from tqdm import tqdm

In [4]:
openai.api_key = input("Provide openai API Key")
os.environ['OPENAI_API_KEY']=openai.api_key

# API Helper Functions

In [5]:
%%capture
import time

alertIncrements = 20
totalCost = 0
totalDavinciTokens = 0
totalGpt3TurboTokens = 0
totalGpt3Turbo16kTokens = 0
totalGpt4Tokens = 0
totalCurieTokens = 0
alertCounter = 0
prompt_generations_log = []

def enumerate_requests(limitLength, lastX):
  for i, (x,y) in enumerate(prompt_generations_log[-lastX:]):
    print("---")
    print("{} : {}, {}".format(i, x, count_gpt2_tokens(y)))
    print(y[:limitLength])

# Function to check if an alertIncrement increase has been reached
def check_alert_increment():
    global alertCounter, totalCost, alertIncrements
    if totalCost >= (alertCounter + 1) * alertIncrements:
        alertCounter += 1
        print("=======================================")
        print("=======================================")
        print(f"Alert: Total cost has reached ${alertCounter * alertIncrements:.2f}")
        print("=======================================")
        print("=======================================")

#Function to count GPT-2 tokens
#Importing GPT-2 tokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

#Count tokens (gpt-3 uses the same tokenizer as gpt-2)
def count_gpt2_tokens(text):
    tokens = tokenizer.tokenize(text)
    return len(tokens)

def accountingCost(prompt, generation, model):
    global totalCost, totalDavinciTokens, totalCurieTokens, totalGpt4Tokens, prompt_generations_log, totalGpt3TurboTokens, totalGpt3Turbo16kTokens
    costPerTokenDavinci = 0.02/1000
    costPerTokenCurie = 0.002/1000
    costPerTokenGpt3TurboPrompt16k = 0.003/1000
    costPerTokenGpt3TurboCompletion16k = 0.004/1000
    costPerTokenGpt3TurboPrompt = 0.0015/1000
    costPerTokenGpt3TurboCompletion = 0.002/1000
    costPerTokenGpt4Prompt = 0.03/1000
    costPerTokenGpt4Generation = 0.06/1000

    promptTokens = count_gpt2_tokens(prompt)
    genTokens = count_gpt2_tokens(generation)
    numTokens = promptTokens + genTokens
    if "curie" in model:
        totalCurieTokens += numTokens
        cost = costPerTokenCurie * numTokens
    if "davinci" in model:
        totalDavinciTokens += numTokens
        cost = costPerTokenDavinci * numTokens
    if "3.5-turbo" in model:
      if "16k" in model:
        totalGpt3Turbo16kTokens += numTokens
        cost = costPerTokenGpt3TurboPrompt16k * promptTokens
        cost += costPerTokenGpt3TurboCompletion16k * genTokens
      else:
        totalGpt3TurboTokens += numTokens
        cost = costPerTokenGpt3TurboPrompt * promptTokens
        cost += costPerTokenGpt3TurboCompletion * genTokens
    if "gpt-4" in model:
        totalGpt4Tokens += numTokens
        cost = costPerTokenGpt4Prompt * promptTokens
        cost += costPerTokenGpt4Generation * genTokens
    totalCost += cost
    prompt_generations_log.append((cost, prompt + generation))

    # Check if an alertIncrement increase has been reached
    check_alert_increment()

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(100), retry=(retry_if_exception_type(openai.error.RateLimitError) | retry_if_exception_type(openai.error.APIError) | retry_if_exception_type(openai.error.ServiceUnavailableError) | retry_if_exception_type(openai.error.APIConnectionError)))
def completion(model, temp, prompt, stopToken="###", maxtokens=200, format="completion"):
    # If the format is "completion"
    if format == "completion":
        response = openai.Completion.create(
            engine=model,
            prompt=prompt,
            max_tokens=maxtokens,
            stop=[stopToken],
            temperature=temp
        )
        generated_text = response['choices'][0]['text']
    # If the format is "chat"
    elif format == "chat":
        # Construct the messages list for the chat format
        messages = [
            {"role": "system", "content": "You are an expert data annotator"},
            {"role": "user", "content": prompt}
        ]

        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            max_tokens=maxtokens,
            stop=[stopToken],
            temperature=temp
        )
        generated_text = response['choices'][0]['message']['content']
    else:
        raise ValueError("Unsupported format. Please use either 'completion' or 'chat'.")

    accountingCost(prompt, generated_text, model)
    return generated_text


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(10), retry=(retry_if_exception_type(openai.error.RateLimitError) | retry_if_exception_type(openai.error.APIError)))
def call_api(model, chunk, stopTokens, temp, max_tokens):
    response = openai.Completion.create(
        model=model,
        prompt=chunk,
        max_tokens=max_tokens,
        stop=stopTokens,
        temperature=temp
    )
    # Call accountingCost for each prompt and generated text
    for choice in response.choices:
        accountingCost(chunk[choice.index], choice.text, model)
    return response

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(10), retry=(retry_if_exception_type(openai.error.RateLimitError) | retry_if_exception_type(openai.error.APIError)))
def batchCompletion(batchSizes, model, temp, prompts, stopTokens=["###"], waitTime=30, tokenLimit=1000):
    results = []
    listPrompts = list(prompts)
    for i in tqdm(range(0, len(prompts), batchSizes)):
        #time.sleep(waitTime)
        chunk = listPrompts[i:i + batchSizes]
        # batching
        responses = [("","")] * len(chunk)
        try:
          response = call_api(
              model=model,
              chunk=chunk,
              stopTokens=stopTokens,
              temp=temp,
              max_tokens=tokenLimit
          )

          # match completions to prompts by index
          for choice in response.choices:
              responses[choice.index] = (chunk[choice.index], choice.text)
        except openai.error.InvalidRequestError:
          print("batch prompting length error")
        results.extend(responses)
    return results

#Text Bottleneck Model Code

Pipeline:

Concept generation
1. Gather most misclassified examples based on current concept set
2. Collect into concept generation prompt and generate concept
3. Improve prompt to avoid common failure cases
4. Trial the concept on a small sample to see if the concept improves accuracy
5. Admit into concept set if it passes

Concept Measurement
1. Collect examples into batches to maximize prompt space
2. Batch generate: snippet extraction, rationalization, final answer

In [7]:
# @title
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics

from random import choice

from concurrent.futures import ThreadPoolExecutor

from sklearn.metrics import accuracy_score, mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import clear_output
import ipywidgets as widgets
import textwrap
import datetime
from tqdm import tqdm


class SupervisedConceptSpace():
  #Initialize supervised concept space
  def __init__(self, train_filepath, test_filepath, is_regression,
               training_configs, name, description, label_guide,
               label_column='label', text_column='text',
               examplesMaxLength = 450, automated=False, modelChoices=None):
    """
    Initializes the SupervisedConceptSpace class.

    Args:
    train_filepath (str): Path to the training data file.
    test_filepath (str): Path to the testing data file.
    is_regression (bool): Whether the task is a regression task.
    training_configs (tuple): Configuration for training (train size limit (int), test size limit (int)).
    name (str): Name of the dataset.
    description (str): Description of the dataset.
    label_guide (dict): Guide for the labels in the dataset.
    label_column (str, optional): Name of the label column in the dataset. Defaults to 'label'.
    text_column (str, optional): Name of the text column in the dataset. Defaults to 'text'.
    examplesMaxLength (int, optional): Maximum length of the examples. Defaults to 450.

    Returns:
    None.
    """
    train_size_limit, test_size_limit = training_configs
    self.train_df_full = pd.read_csv(train_filepath)
    self.test_df_full = pd.read_csv(test_filepath)
    self.train_df = self.train_df_full.sample(min(train_size_limit, len(self.train_df_full)))
    self.test_df = self.test_df_full.sample(min(test_size_limit, len(self.test_df_full)))
    self.name = name
    self.description = description
    self.label_guide = label_guide
    self.label_col = label_column
    self.text_col = text_column
    self.concepts_list = []
    self.concepts_list_unfiltered = []
    self.is_regression = is_regression
    self.examplesMaxLength = examplesMaxLength
    self.flags = []

    #Printing width for interactive printing
    self.printWidth = 150
    self.automated = automated

    #Logging
    self.logs = []
    self.dashboard = {}
    self.dashboard['remaining_iterations'] = -1

    DEFAULT_GENERATION_MODEL = "gpt-4-0613"
    DEFAULT_GENERATION_FORMAT = "chat"

    DEFAULT_MEASUREMENT_MODEL = "gpt-4-0613"
    DEFAULT_MEASUREMENT_FORMAT = "chat"

    if modelChoices:
      DEFAULT_GENERATION_MODEL = modelChoices['DEFAULT_GENERATION_MODEL']
      DEFAULT_GENERATION_FORMAT = modelChoices['DEFAULT_GENERATION_FORMAT']
      DEFAULT_MEASUREMENT_MODEL = modelChoices['DEFAULT_MEASUREMENT_MODEL']
      DEFAULT_MEASUREMENT_FORMAT = modelChoices['DEFAULT_MEASUREMENT_FORMAT']

    self.generation_model = DEFAULT_GENERATION_MODEL
    self.generation_format = DEFAULT_GENERATION_FORMAT
    self.measurement_model = DEFAULT_MEASUREMENT_MODEL
    self.measurement_format = DEFAULT_MEASUREMENT_FORMAT

  # ==== Functions for sampling during Concept Generation ====

  def sample_randomly(self, n_samples_max):
    """
    Samples a specified number of examples from the training data, ensuring a balanced representation of labels.

    Args:
    n_samples_max (int): Maximum number of samples to take.

    Returns:
    samples (DataFrame): Sampled data.
    """
    MAX_TOTAL_TEXT_LENGTH = 4000

    df = self.train_df
    labels = set(df[self.label_col])
    samples_per_label = n_samples_max // len(labels)
    dfs = []

    for label in labels:
        this_label_df = df[df[self.label_col] == label]
        dfs.append(this_label_df.sample(min(samples_per_label, len(this_label_df))))

    samples = pd.concat(dfs)

    #Truncate texts
    samples['text'] = samples['text'].str.slice(0,self.examplesMaxLength)

    # Calculate the total text length
    total_text_length = samples['text'].str.len().sum() * 1.2

    # Remove examples from the label(s) with the highest number of texts until the total text length is within the limit
    while total_text_length > MAX_TOTAL_TEXT_LENGTH:
        label_counts = samples[self.label_col].value_counts()
        max_label = label_counts.idxmax()

        # Get the index of the first occurrence of the max_label
        max_label_index = samples[samples[self.label_col] == max_label].index[0]

        # Remove the row with the max_label
        removed_text_length = len(samples.loc[max_label_index, 'text'])
        samples = samples.drop(max_label_index)

        # Update the total text length
        total_text_length -= removed_text_length * 1.2

    return samples

  def get_neighborhood_mse(_, neighbor_idxs, df, train_pred, label_col):
    """
    Calculates the mean squared error for a given neighborhood of data points.

    Args:
    neighbor_idxs (list): Indices of the neighbors.
    df (DataFrame): DataFrame containing the data.
    train_pred (array): Predicted labels for the training data.
    label_col (str): Name of the label column.

    Returns:
    mse (float): Mean squared error.
    """
    true_labels = df.loc[neighbor_idxs, label_col]
    pred_labels = train_pred[neighbor_idxs]
    mse = mean_squared_error(true_labels, pred_labels)
    return mse

  def get_neighborhood_accuracy(_, neighbor_idxs, df, featureSet, label_col, clf):
    """
    Calculates the accuracy for a given neighborhood of data points.

    Args:
    neighbor_idxs (list): Indices of the neighbors.
    df (DataFrame): DataFrame containing the data.
    featureSet (list): List of features to use for prediction.
    label_col (str): Name of the label column.
    clf (Classifier): Classifier to use for prediction.

    Returns:
    acc (float): Accuracy.
    """
    X = df.loc[neighbor_idxs, featureSet]
    y_true = df.loc[neighbor_idxs, label_col]
    y_pred = clf.predict(X)
    acc = accuracy_score(y_true, y_pred)
    return acc

  def sample_high_error_neighbors(self, train_pred, n_samples_max, n_neighbors, max_length = None):
      """
      Samples a specified number of examples from the training data that have high prediction error.

      Args:
      train_pred (array): Predicted labels for the training data.
      n_samples_max (int): Maximum number of samples to take.
      n_neighbors (int): Number of neighbors to consider.

      Returns:
      samples (DataFrame): Sampled data.
      """
      if "gpt-4" in self.generation_model:
        MAX_TOTAL_TEXT_LENGTH = 8000
      else:
        MAX_TOTAL_TEXT_LENGTH = 4000

      if not max_length:
        max_length = self.examplesMaxLength

      concept_features = [c['Concept Name'] for c in self.concepts_list]
      df_new = self.train_df.copy().reset_index(drop=True)

      knn = NearestNeighbors().fit(np.array(df_new[concept_features]))
      df_new['neighbors'] = pd.Series(knn.kneighbors(np.array(df_new[concept_features]), n_neighbors=n_neighbors)[1].tolist())

      if self.is_regression:
        df_new['neighborhood_loss'] = df_new['neighbors'].apply(self.get_neighborhood_mse, args=(df_new, train_pred, self.label_col))
        idx_order = np.argsort(df_new['neighborhood_loss'])[::-1]
      else:
        df_new['neighborhood_acc'] = df_new['neighbors'].apply(self.get_neighborhood_accuracy, args=(df_new, concept_features, self.label_col, self.clf))
        idx_order = np.argsort(df_new['neighborhood_acc'])
      # Get the top n_samples_max rows
      df_new = df_new.iloc[idx_order[:n_samples_max]]

      #Truncate texts
      df_new['text'] = df_new['text'].str.slice(0,max_length)

      # Calculate the total text length
      total_text_length = df_new['text'].str.len().sum() * 1.2

      # Remove examples from the label(s) with the highest number of texts until the total text length is within the limit
      while total_text_length > MAX_TOTAL_TEXT_LENGTH:
          label_counts = df_new[self.label_col].value_counts()
          max_label = label_counts.idxmax()

          # Get the index of the first occurrence of the max_label
          max_label_index = df_new[df_new[self.label_col] == max_label].index[0]

          # Remove the row with the max_label
          removed_text_length = len(df_new.loc[max_label_index, 'text'])
          df_new = df_new.drop(max_label_index)

          # Update the total text length
          total_text_length -= removed_text_length

      samples = df_new
      if len(samples) < 4:
        self.log("=== Too few examples, trying again with smaller text sizes ===")
        return self.sample_high_error_neighbors(
            train_pred, n_samples_max, n_neighbors, int(max_length * 0.8))
      if samples[self.label_col].nunique() == 1:
        self.flags.append(sample)
      else:
        self.flags.append("clear")
      return samples

  def generate_concept(self, samples, verbose = False):
    """
    Generates a new concept by interacting with the user or using an automated process.

    Args:
    samples (DataFrame): Sampled data.
    verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
    conceptJson (dict): Generated concept.
    """
    try:
      texts = samples[self.text_col].str.slice(0,self.examplesMaxLength).to_list()
      labels = samples[self.label_col].to_list()
      input_type = -1

      #------ Interactive Elements ---------
      self.log("======= Concept Information =======")
      if len(self.concepts_list) > 0:
        featureSet = [c['Concept Name'] for c in self.concepts_list]
        X_train = self.train_df[featureSet]
        y_train = self.train_df['label']

        if self.is_regression:
          model = LinearRegression()
        else:
          model = LogisticRegression()

        model.fit(X_train, y_train)

        coef_dict = {}
        coefs = model.coef_[0] if len(model.coef_.shape) > 1 else model.coef_
        for coef, concept in sorted(zip(coefs, self.concepts_list), key = lambda x: abs(x[0])):
          self.log(f"Feature: {concept['Concept Name']}, Coefficient:{coef}")

          button = widgets.Button(description="Show details")

          # Define what happens when button is clicked
          def on_button_clicked(b, coef=coef, concept=concept):
              self.log(f"Clicked for feature: {concept['Concept Name']}, Coefficient: {coef}")

          # Attach the click event to the button
          button.on_click(on_button_clicked)

          # Display the button
          display(button)
      else:
        self.log("Currently No Concepts")

      self.log("======= Concept Generation =======")


      while input_type not in [0, 1, 2]:
        if self.automated:
          input_type = '0'
        else:
          input_type = input("Write 0 for automated concept generation, Write 1 for suggesting a concept, Write 2 to exit and evaluate")
        if input_type.isdigit():
          input_type = int(input_type)
      if input_type == 0:
        conceptGenerated = self.generate_differences_binary(texts, labels, verbose)
      elif input_type == 1:
        conceptGenerated = self.recommend_concept_assistant(texts, labels, verbose)
      elif input_type == 2:
        self.display_results()
        return None

      cleanedConcept = conceptGenerated.replace("\n","").replace("“","\"").replace("”","\"")
      conceptJson = json.loads(cleanedConcept)

      self.concepts_list_unfiltered.append(conceptJson)
      return conceptJson
    except json.JSONDecodeError:
      self.log("""=== JSON DECODE ERROR in concept generation ===""")
      return self.generate_concept(samples, verbose)

  def generate_differences_binary(self, texts, labels, verbose=False):
    """
    Generates a binary concept based on the differences between the examples.

    Args:
    texts (list): List of text examples.
    labels (list): Corresponding labels for the text examples.
    verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
    response (str): Generated concept.
    """
    # The method first formats the examples and labels into a string, and then
    # uses the OpenAI API to generate a concept based on the examples and labels.
    # The generated concept is returned as a string.

    if not self.is_regression:
      examples = [(text, self.label_guide[str(label)]) for text,label in zip(texts, labels)]
      examples = sorted(examples, key=lambda x: x[1])
      formatted_str = ["text:{}\nrating: {}".format(x,y) for x,y in examples]
      focus_str = f"Focus on concepts that help you distinguish between the following labels from the examples above. The concept should be useful for identifying texts with the following label: {choice([y for x,y in examples])}, you are encouraged to extract keywords and inspiration from the example above."
    else:
      examples = [(text, str(label)) for text,label in zip(texts, labels)]
      examples = sorted(examples, key=lambda x: x[1])
      formatted_str = ["text:{}\nrating: {}".format(x,y) for x,y in examples]
      focus_str = ""
    examples_string = "\n\n".join(formatted_str)


    rejected_concepts = [x for x in concept_tree.concepts_list_unfiltered if
                             x['Concept Name'] not in [y['Concept Name'] for
                             y in concept_tree.concepts_list]]

    rejected_concepts_str = ''
    current_concepts_str = ''
    for i in range(len(self.concepts_list)):
      current_concepts_str += str(i+1)+'. '+self.concepts_list[i]['Concept Name'] + ":" +self.concepts_list[i]['Concept Description']+", possible responses: "+str(self.concepts_list[i]['Possible Responses'])+'\n'
    for i in range(len(rejected_concepts)):
      rejected_concepts_str += str(i+1)+'. '+rejected_concepts[i]['Concept Name'] + ":" +rejected_concepts[i]['Concept Description']+", possible responses: "+str(rejected_concepts[i]['Possible Responses'])+'\n'

    prompt = f"""Concept Feature Engineering Task

Below we are given a text dataset with accompanying labels. Our task is to identify a concept in the text that could be associated with the label. This is because we want to find the main factors that can be used to explain the label.

To do this, we will examine a sample of texts that have different labels so that we can look at the different characteristics that exist for one label and compare it to another. Good concepts are those that separate texts with one label from another.

After looking at these texts and finding a difference, we will define a concept definition JSON
Each full concept definition comes with a concept name, description, question, response set, and response guide. The concept description provides an intuitive overview of the concept. The concept question is our tool for measuring the concept, this will be graded by a human annotator. The possible responses list the possible responses to the question and the response guide provides information on what each rating means. We also include a response mapping to help with data processing.

Below are some examples of concepts for different datasets.

1. A possible concept for a dataset assigning toxicity scores to social media texts
{{"Concept Name": "explicit language",
"Concept Description": "'Explicit language' refers to the use of words, phrases, or expressions that are offensive, vulgar, or inappropriate for general audiences. This may include profanity, obscenities, slurs, sexually explicit or lewd language, and derogatory or discriminatory terms targeted at specific groups or individuals.",
"Concept Question": "What is the nature of the language used in the text?",
"Possible Responses": ["explicit", "strong","non-explicit", "uncertain"],
"Response Guide": {{
"explicit": "The text contains explicit language, such as profanity, obscenities, slurs, sexually explicit or lewd language, or derogatory terms targeted at specific groups or individuals.",
"strong": "The text contains strong language but not explicit language, it may contain terms that some viewers might find mature.",
"non-explicit": "The text is free from explicit language and is appropriate for general audiences.",
"uncertain": "It is difficult to determine the nature of the language used in the text or if any explicit terms are used."
}},
"Response Mapping": {{
"explicit": 2,
"strong":1,
"non-explicit": -1,
"uncertain": 0
}}
}}###

2. A possible concept for evaluating the sentiment of product reviews on ecommerce site
{{"Concept Name": "good build quality",
"Concept Description": "Build quality refers to the craftsmanship, durability, and overall construction of a product. It encompasses aspects such as materials used, design, manufacturing techniques, and attention to detail. A product with good build quality is typically considered to be well-made, sturdy, and long-lasting, while a product with poor build quality may be prone to defects or wear out quickly.",
"Concept Question": "What does the review say about the build quality of the product?",
"Possible Responses": ["positive", "negative", "uncertain", "not applicable"],
"Response Guide": {{
"high": "Review mentions aspects such as well-made, sturdy, durable, high-quality materials, excellent craftsmanship, etc.",
"low": "Review mentions aspects such as poor construction, flimsy, cheap materials, bad design, easily breakable, etc.",
"uncertain": "Review does not mention build quality, the information is ambiguous or vague, or it has both positive and negative aspects mentioned like 'the product is sturdy but uses cheap materials'.",
"not applicable": "The review does not mention the build quality of the product at all."
}},
"Response Mapping": {{
"high": 1,
"low": -1,
"uncertain": 0,
"not applicable": 0
}}
}}###

3. A useful concept for scam detection for emails
{{
"Concept Name": "Extremely generous offer",
"Concept Description": "The concept 'Extremely generous offer' refers to situations where the text describes an offer that seems too good to be true, such as promises of large financial gains, disproportionate rewards, or substantial benefits with seemingly little to no risk or effort required. These can often be indicative of scams or deceptive practices.",
"Concept Question": "What type of offer is described in the text?",
"Possible Responses": ["extremely generous offer", "ordinary offer", "no offer", "uncertain"],
"Response Guide": {{
"extremely generous offer": "The text describes an offer that is disproportionately rewarding or beneficial with seemingly little to no risk or effort. This could include promises of large financial returns with minimal investment, 'free' gifts that require payment information, or rewards that are disproportionate to the effort required.",
"ordinary offer": "The text describes a typical or ordinary offer. For instance, normal sales or discounts, standard business offerings, or fair trades.",
"no offer": "The text does not describe any offer.",
"uncertain": "It is difficult to determine the type of offer described in the text. The text might be vague, ambiguous, or lack sufficient context."
}},
"Response Mapping": {{
"extremely generous offer": 1,
"ordinary offer": -2,
"no offer": -1,
"uncertain": 0
}}
}}

---

In the task, we will generate concepts for the {self.name} dataset

Below is an explanation of the dataset and the labels therein:

Description: {self.description}

Label Guide: {self.label_guide}

Below are some example texts along with their labels. These are examples that the model has a hard time classifying.
---
{examples_string}
---
As a reminder we already have the following concepts which are useful:
{current_concepts_str}
The following concepts have been rejected by the system, avoid making similar ones:
{rejected_concepts_str}

Keeping in mind the pointers above, create a concept below that is distinct from the current set of concepts. Additionally, make sure that all possible responses can be mapped to an integer.
Make sure that the concept is as relevant to the labels in the {self.name} dataset. As the examples shown start to look more similar, we can start being more specific, picking out particular details tied to the label we notice in the examples above.
Below is the description of the dataset.
Description: {self.description}
Additional Notes: {focus_str}
Use strict adherence to the json format. Do not create duplicate concepts.
Definition:"""

    if self.generation_format == "completion":
      formatted_prompt = prompt
      response = completion(model=self.generation_model,
                            temp=1,
                            prompt= formatted_prompt,
                            maxtokens=500,
                            format="completion"
                            )
    elif self.generation_format == "chat":
      formatted_prompt = prompt
      response = completion(model=self.generation_model,
                            temp=1,
                            prompt= formatted_prompt,
                            maxtokens=500,
                            format="chat"
                            )

    if verbose:
      self.log(prompt)
    self.log(response)
    return response

  def measure_concept_prompt(self,concept,text):
    """
    Generates the prompt for measuring a concept.

    Args:
    concept (dict): Concept to measure.
    text (str): Text to measure the concept on.

    Returns:
    prompt (str): Generated prompt.
    """
    # The method first formats the concept and text into a string,
    # and then returns the string as the prompt for measuring the concept.
    conceptString = str({k: concept[k] for k in ('Concept Name', 'Concept Description', 'Concept Question', 'Response Guide')})

    prompt = """Answer the following question about the text below by selecting from the following choices. Before answering the question, extract any potentially relevant snippets of the text that can serve as evidence for each classification. After that, compare the snippets against the response guide to come up with a final decision.

Format your response as a JSON with string keys and string values. Below is an example of a valid JSON response. The JSON contains keys for snippets, thoughts, and answer. End your response with ###
---
Text: Text
Response JSON:{{"snippets": {{
"classification 1" : ["Snippet 1", "Snippet 2", ...],
"classification 2" : ["Snippet 3", "Snippet 4", ...]
...
}},
"thoughts": "In this section, you weigh evidence based on the text and the extracted snippets to come to a final decision with the response guide as a reference. Be as objective as possible and ignore irrelevant information. Focus only on the snippets and avoid making guesses.",
"answer": "An answer from the response guide goes here. In answering the question, ignore irrelevant information and avoid making assumptions."}}###
---
An example of this task being performed can be seen below. Note that the snippet should be as short as possible and be no greater than 10 words.

Concept:
{{
"Concept Name": "good build quality",
"Concept Description": "build quality refers to the craftsmanship, durability, and overall construction of a product. It encompasses aspects such as materials used, design, manufacturing techniques, and attention to detail. A product with good build quality is typically considered to be well-made, sturdy, and long-lasting, while a product with poor build quality may be prone to defects or wear out quickly.",
"Concept Question": "What does the review say about the build quality of the product?",
"Possible Responses": ["high", "low", "uncertain", "not applicable"],
"Response Guide": {{
"high": "Review mentions aspects such as well-made, sturdy, durable, high-quality materials, excellent craftsmanship, etc.",
"low": "Review mentions aspects such as poor construction, flimsy, cheap materials, bad design, easily breakable, etc.",
"uncertain": "Review does not mention build quality, the information is ambiguous or vague, or it has both positive and negative aspects mentioned like 'the product is sturdy but uses cheap materials'.",
"not applicable": "The review does not mention the build quality of the product at all."
}}
}}

Text: "This product has a great design and is really easy to use. It is also very durable."
Response JSON:{{"snippets":{{
"high": ["It is also very durable", "really easy to use"],
"low": [],
"uncertain": []
}},
"thoughts": "Two snippets for high. The first is related to durability which is an aspect of good build quality. The second is related to ease of use which is not related to good build quality. Overall the text describes good build quality.",
"answer": "positive"
}}###
---
Text: "Was excited for it to finally arrive, got here in nice sturdy packaging. Opened it up though and it smelled kind of weird? goes away after a while but otherwise an ok product. Saw some print aberrations it didn't interfere much with use."
Response JSON:{{"snippets":{{
"high": ["got here in nice sturdy packaging"],
"low": ["Saw some print aberrations"],
"uncertain": []
}},
"thoughts": "One snippet for high, one snippet for low. The low snippet comes with a caveat that it 'didn't interfere much with use'. In the positive snippet, 'Sturdy' only refers to the packaging, not the product. We do not have strong enough evidence for a high classification.",
"answer": "low"
}}###
---
Text: "A big fan of the product. Serves me well during workouts but I go through them like hotcakes. Don't expect it to last long compared to other brands but you get what you pay for. It does the job though."
Response JSON:{{"snippets":
{{
"high": ["Serves me well during workouts"],
"low": ["Don't expect it to last long compared to other brands", "I go through them like hotcakes"],
"uncertain": ["you get what you pay for"]
}},
"thoughts": "One high snippet, two low snippets, one uncertain snippet. The high snippet is about utility which is not related to build quality. The first and second low snippet relates to durability, an aspect of build quality.",
"answer": "negative"
}}###
---
Text: "Very disappointing. I was excited to order this but when it arrived I was shocked at how poorly it worked. Deceptive advertising at its finest."
Response JSON:{{"snippets": {{
"high": [],
"low": ["Very disappointing", "shocked at how poorly it worked", "Deceptive advertising"],
"uncertain": []
}},
"thoughts": "Three low spans. The first is related to overall judgment which is irrelevant, the second is related to functionality which is irrelevant, and the third is related to marketing/advertising which is also irrelevant. None are related to build quality.",
"answer": "not applicable"
}}###
---
Perform the task below, keeping in mind to limit snippets to 10 words and ignoring irrelevant information. Return a valid JSON response ending with ###
Concept: {}

Text: {}
Response JSON:""".format(conceptString, text)

    return prompt

  def measure_concept_prompt_batched(self, concept, texts):
    """
    Generates the prompts for measuring a concept on multiple texts.

    Args:
    concept (dict): Concept to measure.
    texts (list): List of texts to measure the concept on.

    Returns:
    batch_sizes (list): Sizes of the batches.
    prompts (list): List of generated prompts.
    """
    # The method first formats the concept and each text into a string, and then
    #returns a list of the strings as the prompts for measuring the concept.
    #It also returns a list of the sizes of the batches.
    conceptString = str({k: concept[k] for k in ('Concept Name', 'Concept Description', 'Concept Question', 'Response Guide')})

    base_prompt = """Answer the following question about the texts below by selecting from the following choices. Before answering the question, extract any potentially relevant snippets of the text that can serve as evidence for each classification. After that, compare the snippets against the response guide to come up with a final decision.

Format your response as a list of JSON objects with string keys and string values. Below is an example of a valid JSON response. Each JSON object contains keys for snippets, thoughts, and answer. End your response with ###
---
Text 1: Text
Text 2: Text
Text 3: Text

Response JSON:[
{{"text": "Text 1", "snippets": {{
"classification 1" : ["Snippet 1", "Snippet 2", ...],
"classification 2" : ["Snippet 3", "Snippet 4", ...]
...
}},
"thoughts": "In this section, you weigh evidence based on the text and the extracted snippets to come to a final decision with the response guide as a reference. Be as objective as possible and ignore irrelevant information. Focus only on the snippets and avoid making guesses.",
"answer": "An answer from the response guide goes here. In answering the question, ignore irrelevant information and avoid making assumptions."}},
{{"text": "Text 2", "snippets": {{
"classification 1" : ["Snippet 1", "Snippet 2", ...],
"classification 2" : ["Snippet 3", "Snippet 4", ...]
...
}},
"thoughts": "...",
"answer": "..."}},
{{"text": "Text 3", "snippets": {{
"classification 1" : ["Snippet 1", "Snippet 2", ...],
"classification 2" : ["Snippet 3", "Snippet 4", ...]
...
}}
]###
---
Below is an example of the task being performed with the concept "build quality":

Concept:
{{
"Concept Name": "good build quality",
"Concept Description": "build quality refers to the craftsmanship, durability, and overall construction of a product. It encompasses aspects such as materials used, design, manufacturing techniques, and attention to detail. A product with good build quality is typically considered to be well-made, sturdy, and long-lasting, while a product with poor build quality may be prone to defects or wear out quickly.",
"Concept Question": "What does the review say about the build quality of the product?",
"Possible Responses": ["high", "low", "uncertain", "not applicable"],
"Response Guide": {{
"high": "Review mentions aspects such as well-made, sturdy, durable, high-quality materials, excellent craftsmanship, etc.",
"low": "Review mentions aspects such as poor construction, flimsy, cheap materials, bad design, easily breakable, etc.",
"uncertain": "Review does not mention build quality, the information is ambiguous or vague, or it has both positive and negative aspects mentioned like 'the product is sturdy but uses cheap materials'.",
"not applicable": "The review does not mention the build quality of the product at all."
}}
}}

Text 1: "This product has a great design and is really easy to use. It is also very durable."
Text 2: "Was excited for it to finally arrive, got here in nice sturdy packaging. Opened it up though and it smelled kind of weird? goes away after a while but otherwise an ok product. Saw some print aberrations it didn't interfere much with use."
Text 3: "A big fan of the product. Serves me well during workouts but I go through them like hotcakes. Don't expect it to last long compared to other brands but you get what you pay for. It does the job though."
Text 4: "Very disappointing. I was excited to order this but when it arrived I was shocked at how poorly it worked. Deceptive advertising at its finest."

Response JSON:[
{{"text": "Text 1", "snippets": {{
"high": ["It is also very durable", "really easy to use"],
"low": [],
"uncertain": []
}},
"thoughts": "Two snippets for high. The first is related to durability which is an aspect of good build quality. The second is related to ease of use which is not related to good build quality. Overall the text describes good build quality..",
"answer": "high"}},

{{"text": "Text 2", "snippets": {{
"high": ["got here in nice sturdy packaging"],
"low": ["Saw some print aberrations"],
"uncertain": []
}},
"thoughts": "One snippet for high, one snippet for low. The low snippet mentions defects in manufacturing. In the high snippet, 'Sturdy' only refers to the packaging, not the product. The balance of evidence leans towards a classification of low.",
"answer": "low"}},

{{"text": "Text 3", "snippets":
{{
"high": ["Serves me well during workouts"],
"low": ["Don't expect it to last long compared to other brands", "I go through them like hotcakes"],
"uncertain": ["you get what you pay for"]
}},
"thoughts": "One high snippet, two low snippets, one uncertain snippet. The high snippet is about utility which is not related to build quality. The low snippet relates to durability, an aspect of build quality.",
"answer": "low"}},

{{"text": "Text 4", "snippets":
{{
"high": [],
"low": ["Very disappointing", "shocked at how poorly it worked", "Deceptive advertising"],
"uncertain": []
}},
"thoughts": "Three low spans. The first is related to overall judgment which is irrelevant, the second is related to functionality which is irrelevant, and the third is related to marketing/advertising which is also irrelevant. None are related to build quality.",
"answer": "uncertain"
}}
]###
---
Perform the task below, keeping in mind to limit snippets to 10 words and ignoring irrelevant information. Return a valid list of JSON objects ending with ###. Note that some texts are truncated due to text length. You must have an output for each piece of text.
Concept: {}

""".format(conceptString)

    base_prompt_token_length = count_gpt2_tokens(base_prompt)

    # Batch texts into groups no greater than 3000 tokens
    batched_texts = []
    batch_sizes = []
    current_batch = ""
    current_batch_allocated_tokens = 0
    current_size = 0

    for text in texts:
        text_tokens = count_gpt2_tokens(text)
        allocation = text_tokens * 2.5 + 200
        if current_batch_allocated_tokens + allocation + base_prompt_token_length < 3500:
            current_size += 1
            current_batch += "Text {}: {}\n".format(current_size, text)
            current_batch_allocated_tokens += allocation
        else:
            if (current_size) > 0:
              batched_texts.append(current_batch)
              batch_sizes.append(current_size)

            current_size = 1
            current_batch = "Text {}: {}\n".format(current_size, text[:2000])
            current_batch_allocated_tokens = allocation

    if current_batch:
        batched_texts.append(current_batch)
        batch_sizes.append(current_size)

    # Add batched texts to the prompts
    prompts = []
    for batch, size in zip(batched_texts, batch_sizes):
        prompt = base_prompt + batch + f"\nLabel for all {size} items:\n Response JSON:[\n{{"
        prompts.append(prompt)


    self.log("batch sizing: {}".format(str(batch_sizes[:5])))
    return batch_sizes, prompts

  def answer_extractor(self, responseText, response_mapping, nullable = False):
    """
    Extracts the answer from the response of the OpenAI API.

    Args:
    responseText (str): Response from the OpenAI API.
    response_mapping (dict): Mapping of responses to scores.
    nullable (bool, optional): Whether to return None for invalid responses. Defaults to False.

    Returns:
    tuple: Extracted answer, score, snippets, and thoughts.
    """
    # The method first cleans the response text and parses it into a JSON object.
    # It then extracts the answer, score, snippets,
    # and thoughts from the JSON object and returns them as a tuple.
    try:
        cleanedResponse = responseText.replace("\n", "").replace("“", "\"").replace("”", "\"")
        jsonResponse = json.loads(cleanedResponse)
        for item in ['snippets', 'thoughts','answer']:
          if item not in jsonResponse.keys():
            return ("na", 0, cleanedResponse, 'Invalid JSON response generated - missing keys')
        answer = jsonResponse['answer']
        score = response_mapping.get(answer, 0)
        return (answer, score, jsonResponse['snippets'], jsonResponse['thoughts'])
    except json.JSONDecodeError:
        if nullable:
          score = None
        else:
          score = 0
        return ("na", 0, cleanedResponse, 'Invalid JSON response generated')

  def answer_extractor_batched(self, responseText, response_mapping, batch_size, nullable = False):
    """
    Extracts the answers from the response of the OpenAI API for multiple texts.

    Args:
    responseText (str): Response from the OpenAI API.
    response_mapping (dict): Mapping of responses to scores.
    batch_size (int): Number of texts in the batch.
    nullable (bool, optional): Whether to return None for invalid responses. Defaults to False.

    Returns:
    list: List of extracted answers, scores, snippets, and thoughts for each text in the batch.
    """
    # The method first cleans the response text and parses it into a list of JSON objects.
    # It then extracts the answer, score, snippets, and thoughts from each JSON object
    # and returns them as a list of tuples.
    try:
        extracted = []
        cleanedResponse = "[{" + responseText.replace("\n", "").replace("“", "\"").replace("”", "\"").replace("],}", "]}")
        jsonResponse = json.loads(cleanedResponse)

        for item in jsonResponse:
          answer = item['answer'].lower()
          try:
            score = response_mapping.get(answer, 0)
          except:
            if nullable:
              score = None
            else:
              score = 0
          extracted.append((answer, score, item['snippets'], item['thoughts']))

        if len(extracted) != batch_size:
          return [("na", 0, cleanedResponse, 'Invalid JSON response generated: Invalid size')]*batch_size
        return extracted
    except json.JSONDecodeError:
        if nullable:
          score = None
        else:
          score = 0
        self.log("Json Decode Error in batch prompt")
        return [("na", 0, cleanedResponse, 'Invalid JSON response generated: JSON Decode error')]*batch_size
    except KeyError:
        if nullable:
          score = None
        else:
          score = 0
        self.log("Key error in batch prompt")
        return [("na", 0, cleanedResponse, 'Invalid JSON response generated: Key Error')]*batch_size

  def measure_concept(self,texts,concept, batching=True):
    """
    Measures a concept by interacting with the OpenAI API.

    Args:
    texts (list): List of texts to measure the concept on.
    concept (dict): Concept to measure.
    batching (bool, optional): Whether to batch the texts. Defaults to True.

    Returns:
    extracted (list): List of extracted answers, scores, snippets, and thoughts for each text.
    """
    # The method first generates the prompts for measuring the concept on the texts.
    # It then sends the prompts to the OpenAI API and extracts the answers, scores,
    # snippets, and thoughts from the responses. The extracted information is returned as a list of tuples.
    MODEL = self.measurement_model
    TEMP = 0
    BATCHSIZES = 20
    truncated_texts = [x[0:self.examplesMaxLength] for x in texts]

    if self.measurement_format == "chat":
      prompts = [self.measure_concept_prompt(concept, text) for text in truncated_texts]
      with ThreadPoolExecutor(max_workers=10) as executor:  # max_workers determines how many threads will be spawned
            responses = list(tqdm(executor.map(lambda p: completion(MODEL, TEMP, p, stopToken="###", maxtokens=1000, format="chat"), prompts), total=len(prompts)))
      extracted = [self.answer_extractor(x, concept['Response Mapping']) for x in responses]
    elif self.measurement_format == "completion":
      if batching:
        batch_sizes, prompts = self.measure_concept_prompt_batched(concept, truncated_texts)
        responses = batchCompletion(BATCHSIZES,MODEL,TEMP,prompts,stopTokens=["###"],tokenLimit=1500, waitTime = 1)
        extracted_list = [self.answer_extractor_batched(x[1], concept['Response Mapping'], batch_sizes[i]) for i,(x,batch_size) in enumerate(zip(responses, batch_sizes))]
        extracted = [item for sublist in extracted_list for item in sublist]
      else:
        prompts = ["{" + self.measure_concept_prompt(concept, text) for text in truncated_texts]
        responses = batchCompletion(BATCHSIZES,MODEL,TEMP,prompts,stopTokens=["###"], waitTime = 1)
        extracted = [self.answer_extractor("{" + x[1], concept['Response Mapping']) for x in responses]

    return extracted

  def label_dataset(self, dataset, concept):
    """
    Labels a dataset with a given concept, skipping already labeled texts.

    Args:
    dataset (DataFrame): Dataset to label.
    concept (dict): Concept to use for labeling.

    Returns:
    None.
    """
    concept_name = concept['Concept Name']
    label_col_name = "{}_label".format(concept_name)

    # Check if the dataset already contains labeled data for this concept
    if label_col_name in dataset.columns:
        # Only proceed with rows where the label column is not already populated
        to_label_idx = dataset[dataset[label_col_name].isna()].index
        subset_to_label = dataset.loc[to_label_idx]
    else:
        subset_to_label = dataset

    # Proceed with labeling only if there are rows to label
    if not subset_to_label.empty:
        answers = self.measure_concept(subset_to_label[self.text_col], concept)

        # Only update the rows that were actually labeled
        dataset.loc[subset_to_label.index, concept_name] = [x[1] for x in answers]
        dataset.loc[subset_to_label.index, label_col_name] = [x[0] for x in answers]
        dataset.loc[subset_to_label.index, "{}_snippets".format(concept_name)] = [x[2] for x in answers]
        dataset.loc[subset_to_label.index, "{}_thoughts".format(concept_name)] = [x[3] for x in answers]



  def add_concept(self, concept, trial_size, threshold, messages=""):
    """
    Adds a concept to the list of concepts.

    Args:
    concept (dict): Concept to add.
    trial_size (int): Size of the trial for evaluating the concept.
    threshold (float): Threshold for accepting the concept.
    messages (str, optional): Messages to display during the process. Defaults to "".

    Returns:
    bool: Whether the concept was added to the list.
    """
    # The method first evaluates the concept on a trial set of examples.
    # It then interacts with the user for feedback and adds the concept to the
    # list if it is accepted. The method returns a boolean
    # indicating whether the concept was added to the list.
    userFeedback = 0
    while userFeedback in [0,1,2]:
      #---- Interactive elements ----
      self.refresh_dashboard()
      self.log(textwrap.fill("Generated Concept: " + json.dumps(concept, indent=2), break_long_words=False,replace_whitespace=False, width=self.printWidth))
      #---- Interactive elements ----
      if self.automated:
        concept = self.modify_concept_assistant(concept)
        userInput = '0'
      else:
        userInput = input("Write 0 to evaluate concept, Write 1 to add this concept to the concepts list, Write 2 to modify, write anything else to reject it: ")
        if userInput.isdigit():
            userFeedback = int(userInput)
        else:
          userFeedback = -1
      if userFeedback == 0:
        concept_name = concept['Concept Name']



        trial_df = self.train_df.sample(trial_size)
        self.trial_df = trial_df
        sampled_indices = trial_df.index
        # Label the trial_df
        self.label_dataset(trial_df, concept)
        # Update the corresponding rows in train_df
        concept_tree.train_df[concept_name] = np.nan
        concept_tree.train_df[f'{concept_name}_label'] = np.nan
        concept_tree.train_df[f'{concept_name}_snippets'] = np.nan
        concept_tree.train_df[f'{concept_name}_thoughts'] = np.nan

        self.train_df.loc[sampled_indices] = trial_df

        self.visualize(trial_df, concept_name)

        featureSet_1 = [c['Concept Name'] for c in self.concepts_list]
        featureSet_2 = featureSet_1.copy()
        featureSet_2.append(concept_name)

        scores = []

        self.log("=== Evaluating concept in trial ===")
        le = LabelEncoder()

        for featureSet in [featureSet_1, featureSet_2]:
          if len(featureSet) == 0:
              if self.is_regression:
                  mean_score = trial_df[self.label_col].mean()
                  scores.append(np.mean((mean_score - trial_df[self.label_col]) ** 2))
              else:
                  most_common_label = trial_df[self.label_col].value_counts().idxmax()
                  scores.append((trial_df[self.label_col] == most_common_label).mean())
          else:
              if self.is_regression:
                  #model = xgb.XGBRegressor(n_estimators = 20, max_depth=3, min_samples_leaf=5, subsample=0.8, gamma=0.5)
                  y_train = trial_df[self.label_col]
                  model = LinearRegression()
                  model.fit(trial_df[featureSet], y_train)
                  preds = model.predict(trial_df[featureSet])
                  scores.append(np.mean((preds - y_train) ** 2))
              else:
                  #model = xgb.XGBClassifier(n_estimators = 20, max_depth=3, min_samples_leaf=5, subsample=0.8, gamma=0.5)
                  y_train = le.fit_transform(trial_df[self.label_col])
                  model = LogisticRegression()
                  model.fit(trial_df[featureSet], y_train)
                  preds = model.predict(trial_df[featureSet])
                  scores.append((preds == y_train).mean())
        self.log("Score 1: {}".format(scores[0]))
        self.log("Score 2: {}".format(scores[1]))

        if self.is_regression:
          delta = -(scores[1] - scores[0])/scores[0]
        else:
          delta = (scores[1] - scores[0])/scores[0]
        self.log("delta: {}".format(delta))

        if self.automated:
          if delta >= threshold:
            userInput = '1'
          else:
            userInput = 'exit'
            userFeedback = 'exit'
        else:
          userInput = input("Write 1 to add this concept to the concepts list, Write 2 to modify, write anything else to reject it: ")
        if userInput.isdigit():
          userFeedback = int(userInput)

      if userFeedback == 1:
        self.log("=== Accepted to concepts list ===")
        self.concepts_list.append(concept)
        self.label_dataset(self.train_df, concept)
        return True
      elif userFeedback == 2:
        concept = self.modify_concept_assistant(concept)


    self.log("=== Rejected from concepts list ===")
    return False


  def score_test_set(self):
    """
    Scores the test set based on the current list of concepts.

    Returns:
    None.
    """
    # The method first measures each concept for each example in the test set.
    # It then calculates an overall score for the test set based on the concept scores.
    featureSet = []
    for concept in self.concepts_list:
      concept_name = concept['Concept Name']
      if concept_name not in self.test_df.columns:
        self.label_dataset(self.test_df, concept)
        featureSet.append(concept_name)
        if self.is_regression:
            #model = xgb.XGBRegressor(n_estimators=100, max_depth=3, min_samples_leaf=3, subsample=0.5, gamma=2)
            model = LinearRegression()
            model.fit(self.train_df[featureSet], self.train_df[self.label_col])
            preds = model.predict(self.test_df[featureSet])
            self.log("Current feature set score: {}".format(np.mean((preds - self.test_df[self.label_col]) ** 2)))
        else:
            #model = xgb.XGBClassifier(n_estimators=100, max_depth=3, min_samples_leaf=3, subsample=0.5, gamma=2)
            model = LogisticRegression()
            model.fit(self.train_df[featureSet], self.train_df[self.label_col])
            preds = model.predict(self.test_df[featureSet])
            self.log("Current feature set accuracy: {}".format((preds == self.test_df[self.label_col]).mean()))
      else:
        featureSet.append(concept_name)

    if self.is_regression:
      model = LinearRegression()
    else:
      model = LogisticRegression()
    model.fit(self.train_df[featureSet], self.train_df[self.label_col])
    featureSet = [c['Concept Name'] for c in self.concepts_list]
    pred_test = model.predict(self.test_df[featureSet])
    self.test_df['preds'] = preds

    if self.is_regression:
      mse = np.mean((pred_test - self.test_df[self.label_col]).pow(2))
      self.log('test mse:',mse)
    else:
      acc = self.clf.score(self.test_df[featureSet], self.test_df[self.label_col])
      self.log('test acc:',acc)


  def iterate(self, trial_size, threshold, remaining_iters = -1, verbose=False):
    """
    Performs one iteration of the concept generation and evaluation process.

    Args:
    trial_size (int): Size of the trial for evaluating the concepts.
    threshold (float): Threshold for accepting the concepts.
    verbose (bool): Whether to print verbose output.

    Returns:
    None.
    """
    # The method first updates the classifier based on the current list of concepts.
    # It then generates a new concept, improves it, evaluates it, and adds it to the
    # list of concepts if it is useful. The method also scores the test set based on
    # the final list of concepts and writes the results to files.

    #update classifier
    featureSet = [c['Concept Name'] for c in self.concepts_list]

    if len(self.concepts_list) == 0:
      samples = self.sample_randomly(20)
    else:
      if self.is_regression:
        #self.clf = xgb.XGBRegressor(n_estimators = 25, max_depth=3, min_samples_leaf=5, subsample=0.8, gamma=0.5)
        self.clf = LinearRegression()
      else:
        #self.clf = xgb.XGBClassifier(n_estimators = 25, max_depth=3, min_samples_leaf=5, subsample=0.8, gamma=0.5)
        self.clf = LogisticRegression()

      self.clf.fit(self.train_df[featureSet], self.train_df[self.label_col])
      pred_train = self.clf.predict(self.train_df[featureSet])
      samples = self.sample_high_error_neighbors(pred_train,20,10)

    if not self.automated:
      userFeedback = -1
      while userFeedback not in [0,1]:
        userInput = input("Write 0 to generate additional concept, Write 1 to finish and evaluate: ")
        if userInput.isdigit():
          userFeedback = int(userInput)
      if userFeedback == 1:
        self.display_results()
        return
      elif userFeedback == 0:
        new_concept = self.generate_concept(samples, verbose)
    else:
      new_concept = self.generate_concept(samples, verbose)

    if new_concept:
      #Prevent name conflict
      if new_concept['Concept Name'] in featureSet:
        new_concept['Concept Name'] += str(len(featureSet))
      #If concept added, add to internal clf
      if self.add_concept(new_concept, trial_size, threshold):
        featureSet = [c['Concept Name'] for c in self.concepts_list]
        if self.is_regression:
          #self.clf = xgb.XGBRegressor(n_estimators = 25, max_depth=3, min_samples_leaf=5, subsample=0.8, gamma=0.5)
          self.clf = LinearRegression()
        else:
          #self.clf = xgb.XGBClassifier(n_estimators = 25, max_depth=3, min_samples_leaf=5, subsample=0.8, gamma=0.5)
          self.clf = LogisticRegression()
        self.clf.fit(self.train_df[featureSet], self.train_df[self.label_col])
        pred_train = self.clf.predict(self.train_df[featureSet])
        if self.is_regression:
          score = np.mean((pred_train - self.train_df[self.label_col]).pow(2))
        else:
          score = self.clf.score(self.train_df[featureSet], self.train_df[self.label_col])
        self.log(f"Internal training set score: {score}")

    if self.automated:
      remaining_iters = remaining_iters - 1
      self.dashboard['remaining_iterations'] = remaining_iters
      if remaining_iters >= 1:
        self.iterate(trial_size, threshold, remaining_iters, verbose)


  def visualize(self,df,concept):
    """
    Visualizes the distribution of a concept in the data.

    Args:
    df (DataFrame): DataFrame containing the data.
    concept (str): Concept to visualize.

    Returns:
    None.
    """
    # The method first extracts examples of each unique value of the concept from
    # the data. It then prints these examples to provide a visual representation
    # of the distribution of the concept in the data.

    value_examples = '\n'
    for i in df[concept].unique():
      value_examples += '----Some examples of '+concept+' with value '+ str(i) + '----\n'
      value_df = df[df[concept]==i]
      examples = value_df.sample(min(5,len(value_df)))
      for ex in examples[self.text_col]:
        value_examples += ex+'\n'
    self.log(value_examples)

  def write_regression_results(self, train_features, test_features):
    """
    Writes the results of the regression model to files.

    Args:
    train_features (DataFrame): Training features.
    test_features (DataFrame): Testing features.

    Returns:
    None.
    """
    # The method first trains a Linear Regression model and an XGBoost model on
    # the training features. It then predicts the labels for the train and test data
    # and calculates the Mean Squared Error for both train and test data. The method
    # saves the accuracies to a CSV file and also saves the coefficients for Linear
    # Regression and feature importances for XGBoost in one file.

    # Get the 'label' columns for train and test data
    train_labels = self.train_df[self.label_col]
    test_labels = self.test_df[self.label_col]

    # Create and train a Linear Regression model
    model_lr = LinearRegression()
    model_lr.fit(train_features, train_labels)

    # Predict the labels for the train and test data
    train_preds_lr = model_lr.predict(train_features)
    test_preds_lr = model_lr.predict(test_features)

    # Calculate the Mean Squared Error for both train and test data
    train_mse_lr = mean_squared_error(train_labels, train_preds_lr)
    test_mse_lr = mean_squared_error(test_labels, test_preds_lr)

    # Train and test XGBoost model
    model_xgb = xgb.XGBRegressor()
    model_xgb.fit(train_features, train_labels)
    train_preds_xgb = model_xgb.predict(train_features)
    test_preds_xgb = model_xgb.predict(test_features)

    # Calculate the Mean Squared Error for both train and test data
    train_mse_xgb = mean_squared_error(train_labels, train_preds_xgb)
    test_mse_xgb = mean_squared_error(test_labels, test_preds_xgb)

    # Save accuracies to a CSV file
    accuracies = pd.DataFrame({
        'model': ['Linear Regression', 'XGBoost'],
        'train_mse': [train_mse_lr, train_mse_xgb],
        'test_mse': [test_mse_lr, test_mse_xgb]
    })
    accuracies.to_csv(f"{self.name}_accuracy.csv", index=False)

    # Save the coefficients for Linear Regression and feature importances for XGBoost in one file
    coefficients_lr = pd.DataFrame({'feature': train_features.columns, 'coefficient': model_lr.coef_, 'model': 'Linear Regression'})
    importances_xgb = pd.DataFrame({'feature': train_features.columns, 'coefficient': model_xgb.feature_importances_, 'model': 'XGBoost'})
    coefficients = pd.concat([coefficients_lr, importances_xgb], ignore_index=True)
    coefficients.to_csv(f"{self.name}_coefficients.csv", index=False)

  def write_classification_results(self, train_features, test_features):
    """
    Writes the results of the classification model to files.

    Args:
    train_features (DataFrame): Training features.
    test_features (DataFrame): Testing features.

    Returns:
    None.
    """
    # The method first trains a Logistic Regression model and an XGBoost model
    # on the training features. It then predicts the labels for the train and
    # test data and calculates the accuracy for both train and test data.
    # The method saves the accuracies to a CSV file and also saves the coefficients
    # for Logistic Regression and feature importances for XGBoost in one file.


    # Get the 'label' columns for train and test data
    train_labels = self.train_df[self.label_col]
    test_labels = self.test_df[self.label_col]

    # Create and train a Logistic Regression model
    model_lr = LogisticRegression()
    model_lr.fit(train_features, train_labels)

    # Predict the labels for the train and test data
    train_preds_lr = model_lr.predict(train_features)
    test_preds_lr = model_lr.predict(test_features)

    # Calculate the accuracy for both train and test data
    train_acc_lr = accuracy_score(train_labels, train_preds_lr)
    test_acc_lr = accuracy_score(test_labels, test_preds_lr)

    # Train and test XGBoost model
    model_xgb = xgb.XGBClassifier()
    model_xgb.fit(train_features, train_labels)
    train_preds_xgb = model_xgb.predict(train_features)
    test_preds_xgb = model_xgb.predict(test_features)

    # Calculate the accuracy for both train and test data
    train_acc_xgb = accuracy_score(train_labels, train_preds_xgb)
    test_acc_xgb = accuracy_score(test_labels, test_preds_xgb)

    # Save accuracies to a CSV file
    accuracies = pd.DataFrame({
        'model': ['Logistic Regression', 'XGBoost'],
        'train_accuracy': [train_acc_lr, train_acc_xgb],
        'test_accuracy': [test_acc_lr, test_acc_xgb]
    })
    accuracies.to_csv(f"{self.name}_accuracy.csv", index=False)

    # Save the coefficients for Logistic Regression and feature importances for XGBoost in one file
    if model_lr.coef_.shape[0] == 1 and len(model_lr.classes_) == 2:
        coef_reshaped = np.vstack((model_lr.coef_, -model_lr.coef_))
    else:
        coef_reshaped = model_lr.coef_

    coefficients_lr = pd.DataFrame(coef_reshaped, columns=train_features.columns)
    coefficients_lr['class'] = model_lr.classes_
    coefficients_lr = coefficients_lr.melt(id_vars=['class'], var_name='feature', value_name='coefficient')
    coefficients_lr['model'] = 'Logistic Regression'

    importances_xgb = pd.DataFrame({'feature': train_features.columns, 'coefficient': model_xgb.feature_importances_, 'model': 'XGBoost'})
    coefficients = pd.concat([coefficients_lr, importances_xgb], ignore_index=True)
    coefficients.to_csv(f"{self.name}_coefficients.csv", index=False)


  def write_results(self):
    """
    Writes the results of the concept generation and evaluation process to files.

    Returns:
    None.
    """
    # The method first saves the training and testing data, the list of concepts,
    # and the scores to CSV files. It then calculates the correlation matrix for
    # the concept scores and saves it to a CSV file. The method also generates a
    # heatmap plot of the correlation matrix and saves it to a PNG file.

    # Save training and testing dataframes
    self.train_df.to_csv(f"{self.name}_train_df.csv", index=False)
    self.test_df.to_csv(f"{self.name}_test_df.csv", index=False)

    # Save the concepts list
    concepts_df = pd.DataFrame(self.concepts_list)
    concepts_df.to_csv(f"{self.name}_concepts.csv", index=False)
    unfiltered_concepts_df = pd.DataFrame(self.concepts_list_unfiltered)
    unfiltered_concepts_df.to_csv(f"{self.name}_concepts_unfiltered.csv", index=False)

    # Prepare train and test data
    concept_features = [c['Concept Name'] for c in self.concepts_list]
    train_features = self.train_df[concept_features]
    test_features = self.test_df[concept_features]

    # Calculate the correlation matrix
    corr_matrix = test_features.corr()

    # Save correlations between the concept scores
    corr_matrix.to_csv(f"{self.name}_correlations.csv")

    # Set up a figure and axes for the plot
    fig, ax = plt.subplots(figsize=(10, 10))

    # Create a custom colormap
    cmap = sns.diverging_palette(220, 20, as_cmap=True)

    # Create the heatmap plot
    sns.heatmap(corr_matrix, cmap=cmap, annot=True, fmt=".2f", linewidths=.5, ax=ax)

    # Set the title for the plot
    ax.set_title("Correlation Matrix Heatmap")

    # Save the correlation plot
    plt.savefig(f"{self.name}_correlation_plot.png")

    # Show the plot
    plt.show()

    # Distribution of concept scores
    train_features.hist(figsize=(15, 15))
    plt.savefig(f"{self.name}_distribution_of_concept_scores.png")
    plt.show()

    if self.is_regression:
        self.write_regression_results(train_features, test_features)
    else:
        self.write_classification_results(train_features, test_features)

  def train(self, n_iters, trial_size, threshold, verbose):
    """
    Trains the concept generation and evaluation process.

    Args:
    n_iters (int): Number of iterations to perform.
    trial_size (int): Size of the trial for evaluating the concepts.
    threshold (float): Threshold for accepting the concepts.
    verbose (bool): Whether to print verbose output.

    Returns:
    None.
    """
    # The method first sets up the concept generation and evaluation process.
    # It then performs a specified number of iterations, with each iteration
    # involving concept generation, improvement, and evaluation. After all
    # iterations are complete, the method scores the test set based on the
    # final list of concepts and writes the results to files.

    input_type = input("Write 0 for interactive mode, Write 1 for automatic mode: ")
    if input_type.isdigit():
      input_type = int(input_type)

    if input_type == 1:
      self.automated = True
      self.dashboard['remaining_iterations'] = n_iters
      self.iterate(trial_size, threshold, n_iters, verbose)
      self.display_results()
    else:
      self.automated = False
      self.iterate(trial_size, threshold, -1, verbose)

  def display_results(self):
    """
    Displays the results of the concept generation and evaluation process.

    Returns:
    None.
    """
    # The method first scores the test set based on the final list of concepts.
    # It then writes the results to files and displays the results in a graphical format.
    self.score_test_set()
    self.write_results()

    results = {}
    results['test'] = pd.read_csv('{}_test_df.csv'.format(self.name))
    results['train'] = pd.read_csv('{}_train_df.csv'.format(self.name))
    results['concepts'] = pd.read_csv('{}_concepts.csv'.format(self.name))
    results['correlations'] = pd.read_csv('{}_correlations.csv'.format(self.name))
    results['coefficients'] = pd.read_csv('{}_coefficients.csv'.format(self.name))
    results['accuracy'] = pd.read_csv('{}_accuracy.csv'.format(self.name))

    display(results['test'].sample(5))
    display(results['concepts'].head(10))
    display(results['correlations'].head(10))
    display(results['coefficients'].tail(20))
    display(results['accuracy'].head(10))

    return results

  def extract_output_list_batched(self, responseText, batch_size):
    """
    Extracts the output list from the response of the OpenAI API for multiple texts.

    Args:
    responseText (str): Response from the OpenAI API.
    batch_size (int): Number of texts in the batch.

    Returns:
    list: List of extracted outputs for each text in the batch.
    """
    # The method first cleans the response text and parses it into a list of outputs. It then returns the list of outputs.

    try:
        extracted = []
        if self.generation_format == 'completion':
          responseText = "[" + responseText
        cleanedResponse = responseText.replace("\n", "").replace("“", "\"").replace("”", "\"").replace("],}", "]}")
        for response in json.loads(cleanedResponse):
            extracted.append(response[1])
        if len(extracted) != batch_size:
            return ["Invalid output generated: Invalid size"]*batch_size
        return extracted
    except json.JSONDecodeError:
        return ["Invalid output generated: Bad generation"]*batch_size


  def modify_concept_assistant(self, concept):
    """
    Modifies a concept based on user feedback.

    Args:
    concept (dict): Concept to modify.

    Returns:
    new_concept (dict): Modified concept.
    """
    # The method first interacts with the user to get feedback on the concept.
    # It then uses the OpenAI API to suggest improvements to the concept based
    # on the user feedback. The improved concept is returned as a dictionary.
    instructionPrompt = lambda x: f"""Concept Improvement Task

We have a concept that needs to be improved. The goal of this task is to identify any issues with the current concept and suggest improvements to make it more valid, clear, well-phrased, and properly formatted in JSON.
In this task, we will return information about any potential problems in the concept along with the improved concept.
Note it is also possible that the concept requires no further improvement (even minor ones), in which case, we will return the original concept with "None" for the other responses.

Consider the following error cases while improving the concept:

1. Lack of validity: Ensure that the responses are mutually exclusive and collectively exhaustive.
- Example of a concept that is not mutually exclusive:
Input JSON:
{{
"Concept Name": "review sentiment",
"Concept Description": "The sentiment expressed towards the product in the review. It could be positive, negative, or neutral.",
"Concept Question": "What is the overall feeling towards the product?",
"Possible Responses": ["positive", "somewhat positive", "negative"],
"Response Guide": {{
"positive": "The reviewer expresses a positive opinion on the product, such as praising its quality, performance, or value.",
"somewhat positive": "The reviewer expresses a somewhat positive opinion on the product, such as mentioning some good aspects but also pointing out some flaws.",
"negative": "The reviewer expresses a negative opinion on the product, such as criticizing its quality, performance, or value."
}},
"Response Mapping": {{
"positive": 1,
"somewhat positive": 0.5,
"negative": -1
}}
}}
User Feedback: The concept above contains non-mutually exclusive responses 'positive' and 'somewhat positive
Response: {{
"Errors": "The concept above contains non-mutually exclusive responses 'positive' and 'somewhat positive'",
"Fix": "We can address this by either combining 'positive' and 'somewhat positive' into a single response or defining clearer distinctions between them.",
"New Concept": {{
"Concept Name": "review sentiment",
"Concept Description": "The sentiment expressed towards the product in the review. It could be positive, negative, or neutral.",
"Concept Question": "What is the overall sentiment expressed towards the product in the review?",
"Possible Responses": ["positive", "negative", "neutral"],
"Response Guide": {{
"positive": "The reviewer expresses a positive opinion on the product, such as praising its quality, performance, or value.",
"negative": "The reviewer expresses a negative opinion on the product, such as criticizing its quality, performance, or value.",
"neutral": "The reviewer does not express a clear positive or negative opinion on the product, or the review contains a mix of positive and negative aspects."
}},
"Response Mapping": {{
"positive": 1,
"negative": -1,
"neutral": 0
}}
}}
}}###

- Example of a concept that is not collectively exhaustive:
Input JSON:
{{
"Concept Name": "product availability",
"Concept Description": "The availability of the product as described in the review.",
"Concept Question": "Is the product available?",
"Possible Responses": ["available", "unavailable"],
"Response Guide": {{
"available": "The reviewer mentions that the product is available, in stock, or easy to find.",
"unavailable": "The reviewer mentions that the product is unavailable, out of stock, or hard to find."
}},
"Response Mapping": {{
"available": 1,
"unavailable": -1
}}
}}
User Feedback: The concept above contains a non-collectively exhaustive response set because it may be possible that a piece of text does not strictly match some criteria in the response guide, such as the availability of the product.
Response: {{
"Errors": "The concept above contains a non-collectively exhaustive response set because it may be possible that a piece of text does not strictly match some criteria in the response guide,",
"Fix": "We can address this by adding a 'uncertain' response to cover cases where the availability is not clearly mentioned, and a 'not applicable' response for cases where the text does not discuss a product.",
"New Concept": {{
"Concept Name": "product availability",
"Concept Description": "The availability of the product as described in the review.",
"Concept Question": "What does the review say about the product's availability?",
"Possible Responses": ["available", "unavailable", "uncertain", "not applicable"],
"Response Guide": {{
"available": "The reviewer mentions that the product is available, in stock, or easy to find.",
"unavailable": "The reviewer mentions that the product is unavailable, out of stock, or hard to find.",
"uncertain": "The review contains mixed information or no information that makes it difficult to determine the availability of the product.",
"not applicable": "The reviewer is not discussing a product or anything else that could be described by this concept."
}},
"Response Mapping": {{
"available": 1,
"unavailable": -1,
"uncertain": 0,
"not applicable": 0
}}
}}
}}###

2. Poor phrasing: Avoid leading questions and provide rich examples in the response guide.
- Example of a leading question:
Input JSON:
{{
"Concept Name": "product cost",
"Concept Description": "The cost of the product as described in the review.",
"Concept Question": "Is the product expensive?",
"Possible Responses": ["expensive", "affordable", "uncertain", "not applicable"],
"Response Guide": {{
"expensive": "The reviewer thinks the product is expensive.",
"affordable": "The reviewer thinks the product is affordable.",
"uncertain": "The reviewer does not provide a clear opinion on the product's cost.",
"not applicable": "The reviewer is not discussing a product or anything else that could be described by this concept."
}},
"Response Mapping": {{
"expensive": 1,
"affordable": -1,
"uncertain": 0,
"not applicable": 0
}}
}}
User Feedback: The concept above contains a leading question and a bad response guide. To fix this, we can simply add more items to the response set.
Response: {{
"Errors": "The concept contains a leading question and a bad response guide",
"Fix": "We can address this by changing the question to be more neutral and asking about the reviewer's description of the product's cost.",
"New Concept": {{
"Concept Name": "product cost",
"Concept Description": "The cost of the product as described in the review, in terms of whether the product is perceived as expensive or affordable.",
"Concept Question": "How does the reviewer describe the cost of the product?",
"Possible Responses": ["expensive", "affordable", "uncertain", "not applicable"],
"Response Guide": {{
"expensive": "The reviewer describes the product as costly, high-priced, or not worth the money.",
"affordable": "The reviewer describes the product as reasonably priced, good value for money, or budget-friendly.",
"uncertain": "The reviewer does not provide a clear opinion or information on the product's cost.",
"not applicable": "The reviewer is not discussing a product or anything else that could be described by this concept."
}},
"Response Mapping": {{
"expensive": 1,
"affordable": -1,
"uncertain": 0,
"not applicable": 0
}}
}}
}}###

In addition to the errors above, some other problems could be:

1. Lack of detail in the response guide. As much as possible, the response guide should contain detailed examples. This issue can be fixed by making the response guide more specific to allow annotators to be more objective about answering the question.
2. All responses should map to integers. They should also be sorted so that the ordering is meaningful. Related responses should be close to each other when mapped to integers.
3. Too many categories

---
Try to make the concept as helpful as possible for prediction task in the dataset, Keep in mind the label guide and the dataset description:
{self.label_guide}
{self.description}
---
Below is the concept for you to improve. Complete the response.
{concept}
User Feedback: {x}
Response:
"""

    if self.automated:
      userComment = "Repair any potential issues if any are detected. Do not repair any issues related to mutual exclusivity/collective exhaustion. Do not change the response set."
    else:
      userComment = input("Write down notes for modifications to concept:")
    userPrompt = instructionPrompt(userComment)
    self.log(userComment)

    try:
      response = completion(self.generation_model,
                              1,
                              userPrompt,
                              maxtokens=2000,
                              format=self.generation_format)
      cleanedResponse = response.replace("\n","").replace("“","\"").replace("”","\"")
      self.log(cleanedResponse)

      responseJson = json.loads(cleanedResponse)

      for item in ['Errors', 'Fix','New Concept']:
          if item not in responseJson.keys():
            self.log("=== JSONDecodeError in response json . Retrying===")
            return self.modify_concept_assistant(concept)

      problem = responseJson['Errors']
      fix = responseJson['Fix']
      new_concept = responseJson['New Concept']

      self.log("=== System Response ===")
      self.log(fix)
      self.log(json.dumps(new_concept, indent=2))

      for item in ['Concept Name', 'Concept Description','Concept Question', 'Possible Responses', 'Response Guide', "Response Mapping"]:
          if item not in new_concept.keys():
            self.log("=== JSONDecodeError in concept modification. Retrying ===")
            return self.modify_concept_assistant(concept)

      return new_concept
    except json.JSONDecodeError:
      self.log("=== JSONDecodeError in concept modification. Retrying ===")
      return self.modify_concept_assistant(concept)


  def recommend_concept_assistant(self, texts, labels, verbose=True):
    """
    Recommends a concept based on the given texts and labels.

    Args:
    texts (list): List of text examples.
    labels (list): Corresponding labels for the text examples.
    verbose (bool, optional): Whether to print verbose output. Defaults to True.

    Returns:
    new_concept (dict): Recommended concept.
    """
    # The method first formats the examples and labels into a string, and then
    # uses the OpenAI API to generate a concept based on the examples and labels.
    # The generated concept is returned as a dictionary.

    examples = [(text, self.label_guide[str(label)]) for text,label in zip(texts, labels)]
    examples = sorted(examples, key=lambda x: x[1])
    formatted_str = ["text:{}\nrating: {}".format(x,y) for x,y in examples]
    focus_str = f"Focus on concepts that help you distinguish between the following labels from the examples above. The concept should be useful for identifying texts with the following label: {choice([y for x,y in examples])}, you are encouraged to extract keywords and inspiration from the example above."
    examples_string = "\n\n".join(formatted_str)

    rejected_concepts = [x for x in concept_tree.concepts_list_unfiltered if
                             x['Concept Name'] not in [y['Concept Name'] for
                             y in concept_tree.concepts_list]]

    rejected_concepts_str = ''
    current_concepts_str = ''
    for i in range(len(self.concepts_list)):
      current_concepts_str += str(i+1)+'. '+self.concepts_list[i]['Concept Name'] + ":" +self.concepts_list[i]['Concept Description']+", possible responses: "+str(self.concepts_list[i]['Possible Responses'])+'\n'
    for i in range(len(rejected_concepts)):
      rejected_concepts_str += str(i+1)+'. '+rejected_concepts[i]['Concept Name'] + ":" +rejected_concepts[i]['Concept Description']+", possible responses: "+str(rejected_concepts[i]['Possible Responses'])+'\n'

    if verbose:
      self.log(f"Description: {self.description}")
      self.log(f"Label Guide: {self.label_guide}")
      self.log(f"Examples: {examples_string}")
      self.log(f"Rejected Concepts: {rejected_concepts_str}")
      self.log(f"Current Concepts: {current_concepts_str}")

    prompt = lambda x: f"""Concept Feature Engineering Task

Below we are given a text dataset with accompanying labels. Our task is to identify a concept in the text that could be associated with the label. This is because we want to find the main factors that can be used to explain the label.

To do this, we will examine a sample of texts that have different labels so that we can look at the different characteristics that exist for one label and compare it to another. Good concepts are those that separate texts with one label from another.

After looking at these texts and finding a difference, we will define a concept definition JSON
Each full concept definition comes with a concept name, description, question, response set, and response guide. The concept description provides an intuitive overview of the concept. The concept question is our tool for measuring the concept, this will be graded by a human annotator. The possible responses list the possible responses to the question and the response guide provides information on what each rating means. We also include a response mapping to help with data processing.

Below are some examples of concepts for different datasets.

1. A possible concept for a dataset assigning toxicity scores to social media texts
{{"Concept Name": "explicit language",
"Concept Description": "'Explicit language' refers to the use of words, phrases, or expressions that are offensive, vulgar, or inappropriate for general audiences. This may include profanity, obscenities, slurs, sexually explicit or lewd language, and derogatory or discriminatory terms targeted at specific groups or individuals.",
"Concept Question": "What is the nature of the language used in the text?",
"Possible Responses": ["explicit", "strong","non-explicit", "uncertain"],
"Response Guide": {{
"explicit": "The text contains explicit language, such as profanity, obscenities, slurs, sexually explicit or lewd language, or derogatory terms targeted at specific groups or individuals.",
"strong": "The text contains strong language but not explicit language, it may contain terms that some viewers might find mature.",
"non-explicit": "The text is free from explicit language and is appropriate for general audiences.",
"uncertain": "It is difficult to determine the nature of the language used in the text or if any explicit terms are used."
}},
"Response Mapping": {{
"explicit": 2,
"strong":1,
"non-explicit": -1,
"uncertain": 0
}}
}}###

2. A possible concept for evaluating the sentiment of product reviews on ecommerce site
{{"Concept Name": "good build quality",
"Concept Description": "Build quality refers to the craftsmanship, durability, and overall construction of a product. It encompasses aspects such as materials used, design, manufacturing techniques, and attention to detail. A product with good build quality is typically considered to be well-made, sturdy, and long-lasting, while a product with poor build quality may be prone to defects or wear out quickly.",
"Concept Question": "What does the review say about the build quality of the product?",
"Possible Responses": ["positive", "negative", "uncertain", "not applicable"],
"Response Guide": {{
"high": "Review mentions aspects such as well-made, sturdy, durable, high-quality materials, excellent craftsmanship, etc.",
"low": "Review mentions aspects such as poor construction, flimsy, cheap materials, bad design, easily breakable, etc.",
"uncertain": "Review does not mention build quality, the information is ambiguous or vague, or it has both positive and negative aspects mentioned like 'the product is sturdy but uses cheap materials'.",
"not applicable": "The review does not mention the build quality of the product at all."
}},
"Response Mapping": {{
"high": 1,
"low": -1,
"uncertain": 0,
"not applicable": 0
}}
}}###

3. A useful concept for scam detection for emails
{{
"Concept Name": "Extremely generous offer",
"Concept Description": "The concept 'Extremely generous offer' refers to situations where the text describes an offer that seems too good to be true, such as promises of large financial gains, disproportionate rewards, or substantial benefits with seemingly little to no risk or effort required. These can often be indicative of scams or deceptive practices.",
"Concept Question": "What type of offer is described in the text?",
"Possible Responses": ["extremely generous offer", "ordinary offer", "no offer", "uncertain"],
"Response Guide": {{
"extremely generous offer": "The text describes an offer that is disproportionately rewarding or beneficial with seemingly little to no risk or effort. This could include promises of large financial returns with minimal investment, 'free' gifts that require payment information, or rewards that are disproportionate to the effort required.",
"ordinary offer": "The text describes a typical or ordinary offer. For instance, normal sales or discounts, standard business offerings, or fair trades.",
"no offer": "The text does not describe any offer.",
"uncertain": "It is difficult to determine the type of offer described in the text. The text might be vague, ambiguous, or lack sufficient context."
}},
"Response Mapping": {{
"extremely generous offer": 1,
"ordinary offer": -2,
"no offer": -1,
"uncertain": 0
}}
}}

---

In the task, we will generate concepts for the {self.name} dataset

Below is an explanation of the dataset and the labels therein:

Description: {self.description}

Label Guide: {self.label_guide}

Below are some example texts along with their labels. These are examples that the model has a hard time classifying.
---
{examples_string}
---
As a reminder we already have the following concepts which are useful:
{current_concepts_str[:3000]}
The following concepts have been rejected by the system, avoid making similar ones:
{rejected_concepts_str[:3000]}

Keeping in mind the pointers above, create a concept below that is distinct from the current set of concepts. Additionally, make sure that all possible responses can be mapped to an integer.
Make sure that the concept is as relevant to the labels in the {self.name} dataset. As the examples shown start to look more similar, we can start being more specific, picking out particular details tied to the label we notice in the examples above.
Below is the description of the dataset.
Description: {self.description}
Below are some additional notes to pay attention to during concept generation
Additional Notes: {x}
Use strict adherence to the json format. Do not create duplicate concepts.
Definition:"""

    newMessages = []
    #---- Interactive Elements ----
    self.refresh_dashboard()
    self.log("==== Concept Generation Prompt ====")
    self.log(textwrap.fill(prompt("[ Guidance Is inserted here ]"), break_long_words=False,replace_whitespace=False, width=self.printWidth))
    #---- Interactive Elements ----
    userComment = input("Write down some guidance notes for concept generation:")
    userPrompt = prompt(userComment)
    self.log(userComment)

    response = completion(self.generation_model,
                            1,
                            userPrompt,
                            maxtokens=2000,
                            format="chat")

    cleanedResponse = response.replace("\n","").replace("“","\"").replace("”","\"")
    self.log(cleanedResponse)

    new_concept = json.loads(cleanedResponse)

    self.log("=== System Response ===")
    self.log(json.dumps(new_concept, indent=2))

    return response

  def refresh_dashboard(self):
    clear_output()
    self.log("===== Concept Bottleck Model =====")
    self.log(f"Current Concepts: {[x['Concept Name'] for x in self.concepts_list]}")
    self.log(f"Total Generated: {len(self.concepts_list_unfiltered)}")
    self.log(f"Total Accepted: {len(self.concepts_list)}")
    self.log(f"Remaining iterations: {self.dashboard['remaining_iterations']}")
    self.log("==================================")

  def log(self, text, tags=(), verbose=True):
    timeStr = str(datetime.datetime.now(datetime.timezone(
        datetime.timedelta(hours=-5))))

    self.logs.append((timeStr, tags, text))
    if verbose:
      print(text)

# Evaluation

## Import Data

In [8]:
#@title
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00


In [9]:
configsDict = {}

In [11]:
#@title
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import re

# ======= Load Cebab =======
cebab = load_dataset("CEBaB/CEBaB")

cebab_train = pd.DataFrame(cebab['train_exclusive'])[[
    'description','review_majority']].rename(columns={
    'description': 'text',
    'review_majority': 'label'
})
cebab_train = cebab_train[cebab_train['label'].apply(lambda x: x.isdigit())]
cebab_train.to_csv('cebab_train.csv', index=False)

cebab_test = pd.DataFrame(cebab['test'])[['description','review_majority']].rename(columns={
    'description': 'text',
    'review_majority': 'label'
})
cebab_test = cebab_test[cebab_test['label'].apply(lambda x: x.isdigit())]
cebab_test.to_csv('cebab_test.csv', index=False)


cebabConfig = {}
cebabConfig['train'] = "cebab_train.csv"
cebabConfig['test'] = "cebab_test.csv"
cebabConfig['type'] = "Regression"
cebabConfig['description'] = 'Restauraunt reviews from opentable'
cebabConfig['class names'] = {"1":"1 Star",
                              "2":"2 Stars",
                              "3":"3 Stars",
                              "4":"4 Stars",
                              "5":"5 Stars"}
configsDict['cebab'] = cebabConfig

## Run Evaluation

### Run Specific Eval

This cell train a TBM given a dataset that exists in the "configsDict" dictionary. As an example for how to load an item into this dictionary, see the cebab example above. The system currently outputs 4 items after training:
- datasetName_test_df.csv - The testing dataset containing all the concept annotations
- datasetName_train_df.csv - The training dataset containing all the concept annotations
- datasetName_concepts.csv - A df of all the concepts included in the bottleneck
- datasetName_concepts_unfiltered.csv - A df of all the concepts generated, including concepts that have not been included.

Some important parameters for this section include
- TRAIN_SIZE_LIMIT - Determines size of the training set for the TBM
- TEST_SIZE_LIMIT  - Determines size of the test set for the TBM
- EXAMPLE_LENGTH_LIMIT  - Determines max character length of examples in prompts
- TRIAL_SIZE - Determines size of training set subset to be used during concept performance testing
- N_ITERS - Number of iterations to perform
- THRESHOLD - Relative performance increase threshold for including a concept in the bottleneck

In [12]:
#@title
import os
import json

#EVAL PARAMETERS
CONFIG_NAME = 'cebab'
TRAIN_SIZE_LIMIT = 50
TEST_SIZE_LIMIT = 50
EXAMPLE_LENGTH_LIMIT = 500
TRIAL_SIZE = 50
N_ITERS = 5
THRESHOLD = 0.01
VERBOSE = True

modelChoices = {}
modelChoices['DEFAULT_GENERATION_MODEL'] = "gpt-3.5-turbo"
modelChoices['DEFAULT_GENERATION_FORMAT'] = "chat"
modelChoices['DEFAULT_MEASUREMENT_MODEL'] = "gpt-3.5-turbo"
modelChoices['DEFAULT_MEASUREMENT_FORMAT'] = "chat"

training_configs = TRAIN_SIZE_LIMIT,TEST_SIZE_LIMIT

print(f"""Confirm the following parameters by typing \"yes\":
Dataset: {CONFIG_NAME}
Concepts to generate: {N_ITERS}
Train size: {TRAIN_SIZE_LIMIT}
Test size: {TEST_SIZE_LIMIT}
Max Text length: {EXAMPLE_LENGTH_LIMIT}
Trial size: {TRIAL_SIZE}
Model choices: {modelChoices}""")
if input("Confirm: ") == "yes":
  clear_output()
  config = configsDict[CONFIG_NAME]
  train_fn = config['train']
  test_fn = config['test']
  task_type = config['type']
  description = config['description']
  label_guide = config['class names']

  if task_type == 'Classification':
    is_regression = False
  elif task_type == 'Regression':
    is_regression = True

  concept_tree = SupervisedConceptSpace(train_fn,test_fn,is_regression,
          training_configs,CONFIG_NAME,description,label_guide,
          examplesMaxLength= EXAMPLE_LENGTH_LIMIT, modelChoices = modelChoices)


  concept_tree.train(N_ITERS, TRIAL_SIZE, THRESHOLD, VERBOSE)
  concept_trees[CONFIG_NAME] = concept_tree

===== Concept Bottleck Model =====
Current Concepts: []
Total Generated: 1
Total Accepted: 0
Remaining iterations: 5
Generated Concept: {
  "Concept Name": "portion size",
  "Concept Description": "Portion size refers to the amount of food served in a restaurant
dish. It can vary from small to large and can impact the overall dining experience. Small portion sizes may leave customers unsatisfied, while large
portion sizes may be overwhelming or lead to wastage. The concept of portion size can influence the rating given to a restaurant.",
  "Concept
Question": "What is the size of the portions in the restaurant dish?",
  "Possible Responses": [
    "small",
    "medium",
    "large",
"uncertain"
  ],
  "Response Guide": {
    "small": "The portion size of the dish is small, which is not sufficient to satisfy the customer.",
"medium": "The portion size of the dish is moderate, neither small nor large.",
    "large": "The portion size of the dish is large, which is generous
and satisfying

 40%|████      | 20/50 [09:15<13:53, 27.79s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-7-ce613d8ab4b4>", line 829, in measure_concept
    responses = list(tqdm(executor.map(lambda p: completion(MODEL, TEMP, p, stopToken="###", maxtokens=1000, format="chat"), prompts), total=len(prompts)))
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1182, in __iter__
    for obj in iterable:
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 621, in result_iterator
    yield _result_or_cancel(fs.pop())
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 319, in _result_or_cancel
    return fut.result(timeout)
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 453, in result
    self._condition.wait(timeout)
  File "/usr/lib/python3.10/threading.py", line 320, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/intera

TypeError: ignored